## Presidents and Income
### An intro to python
In this notebook we'll use historical data to look at the relationships between presidential parties and income growth.

The first step is to import the libraries we need for the analysis.

In [1]:
import pandas as pd #data science library for dataframes and data manipulation
import os #operating system library to set the working directory

In [2]:
os.chdir("/Users/bi6fgjk/Downloads/hist_income_by_quintile-main/") 

Pandas has a read_csv function that can read files directly from Github. This is convenient since it saves people from needing to download the files in order to follow along. 

I made a csv of table F-1: https://www.census.gov/data/tables/time-series/demo/income-poverty/historical-income-families.html and saved it to Github to make it available for download. 

All values are in 2019 dollars as provided by the Census

In [9]:
hist_df = pd.read_csv("https://raw.githubusercontent.com/natekratzer/hist_income_by_quintile/main/hist_income47.csv")
hist_df.tail(8)

,Year,20,40,60,80,95
67,1954,"16,759","29,479","39,907","54,676","86,645"
68,1953,"17,976","30,507","40,936","55,130","85,738"
69,1952 (7),"17,370","28,098","38,014","51,416","79,996"
70,1951,"16,826","27,432","36,289","49,548","77,652"
71,1950,"15,441","26,550","35,335","49,112","80,088"
72,1949 (6),"14,523","24,811","33,471","47,298","75,921"
73,1948,"15,385","25,295","33,773","47,114","76,722"
74,1947 (5),"15,903","25,662","34,799","49,377","81,043"
